In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#setting
prompt_style = 'AutoCOT2'
data_count = 0
example_count = 10


In [ ]:
# Configs
import config 

In [ ]:
# Run generator.js
import subprocess

params = ["200", "200"] # Enter the number of data. each prameter indicate the number of small, large map.

subprocess.run(["node", "generator.js"] + params)

In [ ]:
# Run placer.py
import placer

maps = placer.load_maps()
for map in maps:
    placer.assign_parameters(map, enemy_density=0.05, cohesion=0.3, treasure_density=0.01, range_multiplier=2, boss=True)
    placer.modify_map(map, group_min_dist=10, flag_try_count=50, enemy_sparsity=3)
placer.save_maps(maps)

In [ ]:
# Run labeler.py
import labeler

labeler.label(placed_path=config.PLACED_PATH, labelled_path=config.LABELLED_PATH, file_count=4, difficulty_curve_interval=5)

In [ ]:
import utility
import numpy as np

param_name_list = ["map_size", "enemy_count", "treasure_count", "room_count"]

labelled_data_list = utility.load_json_files(config.LABELLED_PATH)

mean_dict = dict()
std_dict = dict()

for param_name in param_name_list:
    value_list = np.array([data["params"][param_name] for data in labelled_data_list])
    mean_dict[param_name] = np.mean(value_list, axis=0)
    std_dict[param_name] = np.std(value_list, axis=0)

print(mean_dict)
print(std_dict)

In [ ]:
# Generate preprocessed
from preprocessed_data_generator import generate_preprocessed_data

generate_preprocessed_data(data_count, example_count, prompt_style)

In [ ]:
# Run comparator.py
import comparator

comparator.compare(preprocessed_path=config.PREPROCESSED_PATH, compared_path=config.COMPARED_PATH, file_count=83)

In [ ]:
# Calculate statistics
import statistician

diff_param_name_list = [
    "map_size",
    "room_count",
    "enemy_count",
    "treasure_count",
]

after_param_name_list = ["playability", "other_ASCII_count", "empty_validation"]

abs_diff_mean, abs_diff_std = statistician.calc_abs_diff_mean_std(param_name_list=diff_param_name_list)
after_mean, after_std = statistician.calc_after_mean_std(param_name_list=after_param_name_list)

In [ ]:
# Draw Graph
mean_dict = abs_diff_mean | after_mean
std_dict = abs_diff_std | after_std

mean_dict["map_width"] = mean_dict["map_size"][0]
mean_dict["map_height"] = mean_dict["map_size"][1]

std_dict["map_width"] = std_dict["map_size"][0]
std_dict["map_height"] = std_dict["map_size"][1]

del mean_dict["map_size"]
del std_dict["map_size"]

print(mean_dict)
print(std_dict)
statistician.draw_graph(mean_dict, std_dict)

In [ ]:
# Calculate novelty and diversity
novelty = statistician.calc_novelty()
print(novelty)

In [ ]:
# Calculate diversity
diversity = statistician.calc_diversity()
print(diversity)

In [ ]:
import labeler

data_list= labeler.load_folder(path=config.PREPROCESSED_PATH, file_count=84)
data_count = 0

for i, data_i in enumerate(data_list):
    data_count += len(data_i["map_list"])

print(data_count)